# Part 1: Load Data

In this section, we will load the appropriate data and select the columns we will use

In [1]:
# Import necessary libraries
from azure.storage.blob import BlobServiceClient
import pandas as pd
import io

# Initialize a BlobServiceClient
sas_token = "sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupiytfx&se=2024-05-13T13:01:11Z&st=2024-04-13T05:01:11Z&spr=https,http&sig=X362UUEwAfkGDCBr2G0ZOc2SX%2FLg9LOddGvPHZ%2Fff00%3D"  # Ensure this is URL-encoded
blob_service_client = BlobServiceClient(account_url="https://sadukedatauseprod.blob.core.windows.net", credential=sas_token)

# Function to list blobs in a container
def list_blobs(container_name):
    container_client = blob_service_client.get_container_client(container=container_name)
    return [blob.name for blob in container_client.list_blobs()]

# Function to load data from a specific blob within a container
def load_data(container_name, blob_name):
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    blob_content = blob_client.download_blob().readall()
    blob = pd.read_csv(io.StringIO(blob_content.decode('utf-8')))
    print(blob_name.split('.')[0])
    display(blob.head(0))
    print()
    return blob

CONTAINER='mimiciii'


In [2]:
# Download datasets and print columns
# ICD = load_data(CONTAINER, 'DIAGNOSES_ICD.csv')
# Notes=load_data(CONTAINER,"NOTEEVENTS.csv")
# Micro = load_data(CONTAINER, 'MICROBIOLOGYEVENTS.csv')
# Admissions = load_data("mimiciii","ADMISSIONS.csv")
# Patients = load_data(CONTAINER, 'PATIENTS.csv')
# Abx = pd.read_csv('Abx.csv')

print('ABX')
display(Abx.head(0))

DIAGNOSES_ICD


,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE


/tmp/ipykernel_130366/3464553357.py:19: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  blob = pd.read_csv(io.StringIO(blob_content.decode('utf-8')))


NOTEEVENTS


,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT


...


PATIENTS


,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG



ABX


/tmp/ipykernel_130366/4018074662.py:7: DtypeWarning: Columns (11,16) have mixed types. Specify dtype option on import or set low_memory=False.
  Abx = pd.read_csv('Abx.csv')


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE


In [3]:
# Select Columns and set columns to strings
ICD = ICD[['HADM_ID', 'SUBJECT_ID']].astype(str)

Discharges = Notes[Notes['CATEGORY']=="Discharge summary"]
Discharges[['HADM_ID']] = Discharges[['HADM_ID']].astype(int)
Discharges = Discharges[['HADM_ID', 'SUBJECT_ID', 'CHARTDATE', 'TEXT']].astype(str)
display(Discharges)

BloodCx = Micro[Micro['SPEC_TYPE_DESC']=='BLOOD CULTURE']
BloodCx = BloodCx[~BloodCx['INTERPRETATION'].isna()]
BloodCx = BloodCx[['HADM_ID']].astype(str)

Admissions = Admissions[['HADM_ID', 'SUBJECT_ID', 'ADMITTIME', 'INSURANCE', 'ETHNICITY', 'DIAGNOSIS']].astype(str)

Patients = Patients[['SUBJECT_ID', 'GENDER']].astype(str)

Abx = Abx[['HADM_ID', 'ENDDATE']]
Abx['HADM_ID'] = Abx['HADM_ID'].astype(str)


/tmp/ipykernel_130366/872148071.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Discharges[['HADM_ID']] = Discharges[['HADM_ID']].astype(int)


,HADM_ID,SUBJECT_ID,CHARTDATE,TEXT
0,167853,22532,2151-08-04,Admission Date: [**2151-7-16**] Dischar...
1,107527,13702,2118-06-14,Admission Date: [**2118-6-2**] Discharg...
2,167118,13702,2119-05-25,Admission Date: [**2119-5-4**] D...
3,196489,13702,2124-08-18,Admission Date: [**2124-7-21**] ...
4,135453,26880,2162-03-25,Admission Date: [**2162-3-3**] D...
...,...,...,...,...
60411,135672,93924,2104-06-14,"Name: [**Known lastname 18321**],[**Known fir..."
60412,155710,17782,2112-03-31,"Name: [**Known lastname 2706**], [**Known fir..."
60413,183951,96333,2197-06-18,Name: [**Known lastname 18357**]-[**Known las...
60414,169165,66717,2129-08-14,"Name: [**Known lastname **],[**Known firstnam..."


In [4]:
# Delete Larger Datasets if needed (can set to false)
if False:
    del ICD
    del Notes
    del Micro
    del Patients

# Part 2: Synthesize three sections

In this section, we will combine the data needed into the three datasets that describe the ways we will determine sepsis. Those are:
* ICD diagnoses
* Inferred ICD diagnoses _(from NLP)_
* Combination of Lab data and Antibiotic Prescriptions

## Part 2.1: ICD Data

In [5]:
Sepsis_ICDs = ICD[ICD.ICD9_CODE.isin(['99591', '99592'])] # ICD = Sepsis
Sepsis_ICDs = Sepsis_ICDs[['SUBJECT_ID', 'HADM_ID', 'ICD9_CODE']].astype(str)
Sepsis_ICDs['Sepsis by ICD'] = True
Sepsis_ICDs.dtypes

SUBJECT_ID       object
HADM_ID          object
ICD9_CODE        object
Sepsis by ICD      bool
dtype: object

## Part 2.2: Discharge Notes (NLP)

In [6]:
# Matt - this is where the openai shenannigans goes
Discharges['Sepsis by note'] = True
Discharges['HADM_ID']

0        167853
1        107527
2        167118
3        196489
4        135453
          ...  
60411    135672
60412    155710
60413    183951
60414    169165
60415    157100
Name: HADM_ID, Length: 59652, dtype: object

## Part 2.3: Lab values and Antibiotics

We used two critera to determine patients with sepsis:
* Must have had at least one positive blood culture during their admission
* Must have continued on antibiotics at least 48 hrs from admission

In [7]:
# Merge data
BloodAmissions = pd.merge(BloodCx, Admissions, on="HADM_ID", how="inner")
SepsisTx = pd.merge(Abx, BloodAmissions, on="HADM_ID", how="inner")

# Filter antibiotics and drop duplicates
SepsisTx['ABXRXHOURS'] = (pd.to_datetime(SepsisTx['ENDDATE']) - pd.to_datetime(SepsisTx['ADMITTIME'])).dt.total_seconds()//3600
SepsisTx=SepsisTx[SepsisTx['ABXRXHOURS'] >=48]
SepsisTx = SepsisTx.drop_duplicates('HADM_ID', keep='first')
SepsisTx = SepsisTx[['HADM_ID', 'SUBJECT_ID']].astype(str)
SepsisTx['Sepsis by Labs'] = True 

SepsisTx

,HADM_ID,SUBJECT_ID,Sepsis by Labs
4,197487,85071,True
84,108084,68275,True
148,154851,43320,True
420,157686,18254,True
720,148350,76665,True
...,...,...,...
488952,153102,99384,True
489026,166271,31539,True
489029,155547,19375,True
489037,155316,20356,True


# Part 3: Working with Combined Data

In [8]:
# Merge the data
merged_data = pd.merge(Sepsis_ICDs, Discharges, on='HADM_ID', how='left', suffixes=('_ICD', '_notes'))#outer, once NLP works
All_Sepsis = pd.merge(merged_data, SepsisTx, on='HADM_ID', how='outer')
# Handle duplicated SUBJECT_ID
All_Sepsis['SUBJECT_ID'] = (
    All_Sepsis['SUBJECT_ID']
            .combine_first(All_Sepsis['SUBJECT_ID_ICD'])
            .combine_first(All_Sepsis['SUBJECT_ID_notes'])
)
All_Sepsis = All_Sepsis.drop(columns=['SUBJECT_ID_ICD', 'SUBJECT_ID_notes'])
All_Sepsis[['Sepsis by ICD', 'Sepsis by note', 'Sepsis by Labs']] = All_Sepsis[['Sepsis by ICD', 'Sepsis by note', 'Sepsis by Labs']].fillna(False)
All_Sepsis

,HADM_ID,ICD9_CODE,Sepsis by ICD,CHARTDATE,TEXT,Sepsis by note,SUBJECT_ID,Sepsis by Labs
0,164853,99592,True,2133-12-01,Admission Date: [**2133-11-13**] ...,True,117,False
1,138376,99592,True,2166-02-01,Unit No: [**Numeric Identifier 13036**]\nAdmi...,True,124,True
2,172056,99591,True,2143-03-18,Admission Date: [**2143-3-3**] Discharg...,True,64,False
3,112077,99591,True,2167-07-30,Admission Date: [**2167-7-25**] ...,True,85,False
4,111970,99592,True,2135-02-08,Admission Date: [**2135-1-30**] ...,True,21,True
...,...,...,...,...,...,...,...,...
7100,197258,NaN,False,NaN,NaN,False,19213,True
7101,182203,NaN,False,NaN,NaN,False,23709,True
7102,187610,NaN,False,NaN,NaN,False,26918,True
7103,155316,NaN,False,NaN,NaN,False,20356,True


## Part 3.1: Demographic chart

In [9]:
demo_cols = ['ETHNICITY', 'INSURANCE', 'GENDER']
temp = pd.merge(All_Sepsis, Patients, on='SUBJECT_ID', how='left')
Demographics = pd.merge(temp, Admissions[['HADM_ID', 'ETHNICITY', 'INSURANCE']], on='HADM_ID')
# Handle missing data 
Demographics[demo_cols] = Demographics[demo_cols].fillna('UNKNOWN')

#Group Ethnicities
Demographics['ETHNICITY'] = Demographics['ETHNICITY'].str.replace(r'([A-Z]+).*', r'\1', regex=True)
Demographics['ETHNICITY'] = Demographics['ETHNICITY'].str.replace(r'(MULTI|PORTUGUESE|MIDDLE|CARIBBEAN|AMERICAN|NATIVE|SOUTH)', 'OTHER', regex=True)
Demographics['ETHNICITY'] = Demographics['ETHNICITY'].str.replace(r'(UNABLE|PATIENT)', 'UNKNOWN', regex=True)
Demographics['ETHNICITY'].value_counts()


ETHNICITY
WHITE       5185
BLACK        751
UNKNOWN      512
HISPANIC     235
ASIAN        213
OTHER        209
Name: count, dtype: int64

In [10]:
import tableone
tableone.tableone(
    Demographics,
    columns= [
        'INSURANCE',
        'ETHNICITY',
        # 'DIAGNOSIS',
        'GENDER',
        'Sepsis by Labs',
        'Sepsis by note',
        'Sepsis by ICD',
    ]
)

Missing      Overall
n                                                7105
INSURANCE, n (%)      Government       0    144 (2.0)
                      Medicaid              611 (8.6)
                      Medicare            4472 (62.9)
                      Private             1837 (25.9)
                      Self Pay               41 (0.6)
ETHNICITY, n (%)      ASIAN            0    213 (3.0)
                      BLACK                751 (10.6)
                      HISPANIC              235 (3.3)
                      OTHER                 209 (2.9)
                      UNKNOWN               512 (7.2)
                      WHITE               5185 (73.0)
GENDER, n (%)         F                0  3109 (43.8)
                      M                   3996 (56.2)
Sepsis by Labs, n (%) False            0  4136 (58.2)
                      True                2969 (41.8)
Sepsis by note, n (%) False            0  1654 (23.3)
                      True                5451 (76.7)
Sepsis by ICD, n (%)  False            0  1513 (21.3)
                      True                5592 (78.7)

In [11]:
tableone.tableone(
    Demographics,
    columns= [
        'INSURANCE',
        'ETHNICITY',
        # 'DIAGNOSIS',
        'GENDER',
        'Sepsis by Labs',
        'Sepsis by note',
        'Sepsis by ICD',
    ],
    groupby=[
        'ETHNICITY',
    ]
)

Grouped by ETHNICITY                                                                                      
                                              Missing      Overall       ASIAN       BLACK    HISPANIC       OTHER     UNKNOWN        WHITE
n                                                             7105         213         751         235         209         512         5185
INSURANCE, n (%)      Government                    0    144 (2.0)    14 (6.6)    22 (2.9)    12 (5.1)     6 (2.9)    16 (3.1)     74 (1.4)
                      Medicaid                           611 (8.6)   36 (16.9)  123 (16.4)   45 (19.1)   36 (17.2)    49 (9.6)    322 (6.2)
                      Medicare                         4472 (62.9)  119 (55.9)  484 (64.4)  130 (55.3)   94 (45.0)  285 (55.7)  3360 (64.8)
                      Private                          1837 (25.9)   43 (20.2)  120 (16.0)   45 (19.1)   66 (31.6)  155 (30.3)  1408 (27.2)
                      Self Pay                            41 (0.6)     1 (0.5)     2 (0.3)     3 (1.3)     7 (3.3)     7 (1.4)     21 (0.4)
GENDER, n (%)         F                             0  3109 (43.8)   89 (41.8)  418 (55.7)   76 (32.3)   85 (40.7)  209 (40.8)  2232 (43.0)
                      M                                3996 (56.2)  124 (58.2)  333 (44.3)  159 (67.7)  124 (59.3)  303 (59.2)  2953 (57.0)
Sepsis by Labs, n (%) False                         0  4136 (58.2)  129 (60.6)  413 (55.0)  117 (49.8)  108 (51.7)  290 (56.6)  3079 (59.4)
                      True                             2969 (41.8)   84 (39.4)  338 (45.0)  118 (50.2)  101 (48.3)  222 (43.4)  2106 (40.6)
Sepsis by note, n (%) False                         0  1654 (23.3)   37 (17.4)  181 (24.1)   61 (26.0)   56 (26.8)  145 (28.3)  1174 (22.6)
                      True                             5451 (76.7)  176 (82.6)  570 (75.9)  174 (74.0)  153 (73.2)  367 (71.7)  4011 (77.4)
Sepsis by ICD, n (%)  False                         0  1513 (21.3)   30 (14.1)  174 (23.2)   54 (23.0)   54 (25.8)  130 (25.4)  1071 (20.7)
                      True                             5592 (78.7)  183 (85.9)  577 (76.8)  181 (77.0)  155 (74.2)  382 (74.6)  4114 (79.3)

## Part 3.2: database sizes

In [ ]:
print(f"There are {len(ICD)} patients with ICD codes."})
print(f"There are {len(Sepsis_ICDs} patients with ICD codes indicating sepsis.")
print()
print(f"There are {len(Discharges)} discharge notes.")
print(f"The Large Language Model identified that XX of them predicted Sepsis.")
print()
print(f"Of {len(BloodCx)} positive blood cultures and {len(Abx)} antibiotic prescriptions,")
print(f"{len(SepsisTx)} were indicated that